# 读取数据

In [2]:
import pickle
road_zip={}
road_edges={}
node_dict={}
node_dict_amap={}
node_poi={}
with open('../data/road_zip.pkl','rb') as f:
    road_zip=pickle.load(f)
with open('../data/road_edges.pkl','rb') as f:
    road_edges=pickle.load(f)
with open('../data/map_nodes.pkl','rb') as f:
    node_dict=pickle.load(f)
with open('../data/map_nodes_amap.pkl','rb') as f:
    node_dict_amap=pickle.load(f)
with open('../data/POI.pkl','rb') as f:
    node_poi=pickle.load(f)

# 数据原子化

生成.dyna文件

In [3]:
import numpy as np
import datetime
import csv

In [4]:
node_roads_aggre={}
for road,edges in road_edges.items():
    node_list=list(set([edge[0] for edge in edges]+[edge[1] for edge in edges]))
    for node in node_list:
        if node in node_roads_aggre:
            node_roads_aggre[node].append(road)
        else:
            node_roads_aggre[node]=[road]
            
node_speed_aggre={}
zip_shape=np.array(list(road_zip.values())).shape

for node,roads in node_roads_aggre.items():
    mean_speed=np.zeros((zip_shape[1],zip_shape[2]))
    for road in roads:
        mean_speed+=np.array(road_zip[road][:,:,1])
    mean_speed/=len(roads)
    node_speed_aggre[node]=mean_speed

print('data length',len(node_speed_aggre))  
with open(f'../data/atom/TGESHD.dyna','w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['dyna_id','type','time','entity_id','traffic_speed'])
    start_day=datetime.datetime.strptime('2022-01-25T05:00:00Z','%Y-%m-%dT%H:%M:%SZ')
    no = 0
    for node,speed in node_speed_aggre.items():
        for day in range(speed.shape[0]):
            for time in range(speed.shape[1]):
                delta = datetime.timedelta(days=day,minutes=time*5)
                csv_writer.writerow([
                    no,
                    'state',
                    (start_day+delta).strftime("%Y-%m-%dT%H:%M:%SZ"),
                    node,
                    speed[day][time]
                ])
                no+=1
    f.close()

data length 4499


生成.geo文件

In [5]:
node_poi_code={}
for node in node_poi:
    if node in node_roads_aggre:
        node_poi_code[node]=list(set([poi['typecode'][0:2] for poi in node_poi[node]]))

with open(f'../data/atom/TGESHD.geo','w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['geo_id','type','coordinates','Food','Shopping','Daily','Sports','Medical','Accommodation','Tourist','House','Government','Education','Transportation','Finance','Enterprises'])
    for node,pois in node_poi_code.items():
        csv_writer.writerow([
            node,
            'Point',
            '['+node_dict_amap[node][0]+','+node_dict_amap[node][1]+']',
            1 if '05' in pois else 0,
            1 if '06' in pois else 0,
            1 if '07' in pois else 0,
            1 if '08' in pois else 0,
            1 if '09' in pois else 0,
            1 if '10' in pois else 0,
            1 if '11' in pois else 0,
            1 if '12' in pois else 0,
            1 if '13' in pois else 0,
            1 if '14' in pois else 0,
            1 if '15' in pois else 0,
            1 if '16' in pois else 0,
            1 if '17' in pois else 0
        ])
    f.close()
print('data length',len(set(node_poi_code)))

data length 4499


生成rel文件

In [6]:
with open(f'../data/atom/TGESHD.rel','w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['rel_id','type','origin_id','destination_id','road_name','road_direction','weight'])
    no=0
    for road,edges in road_edges.items():
        for edge in edges:
            csv_writer.writerow([
                no,
                'geo',
                edge[0],
                edge[1],
                road[0],
                road[1],
                1
            ])
            no+=1